# Setup
Import libraries, etc.

In [2]:
! pip install evaluate bert_score rouge_score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.5 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=c93928d4ee1e2993d8f9475fd0dae76b903cb2840bf92521efd79951d3bc47d3
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [3]:
import pandas as pd
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch
from evaluate import load
from tqdm.notebook import tqdm
import numpy as np

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


In [7]:
rouge_score = load('rouge')
bert_score = load("bertscore")

# Dataset
Hasil scraping review dari Zomato

In [7]:
df = pd.read_csv('/kaggle/input/zomatoreview/zomato_reviews.csv')
df.head()

,resto_name,review_text,timestamp
0,Cé La Vie Kitchen & Bar,"They are offering false offers in Zomato gold,...",5 jam yang lalu
1,Cé La Vie Kitchen & Bar,The ambience and hygiene was just not it. The ...,15 jam yang lalu
2,Cé La Vie Kitchen & Bar,Very nice ambience with amazing vibesPrem kuma...,16 jam yang lalu
3,Cé La Vie Kitchen & Bar,They denied to take payment from Zomato gold ....,17 jam yang lalu
4,Cé La Vie Kitchen & Bar,It was good experience at sanam,19 jam yang lalu


# Eksperimen
Akan membandingkan *performance* dari 3 buah model untuk pekerjaan summarization teks. Generation menggunakan prompt zero-shot.

Model yang digunakan antara lain adalah
1. facebook/bart-large-cnn
2. google-t5/t5-small
3. google/bigbird-pegasus-large-arxiv

# Prompt

In [100]:
# base_prompt = "Summarize the following content to show the important part of the text and it's reason:"
# base_prompt = """Summarize: This service was good. The way that they handles with the customers is a plus point for me. They were very polite and helpful
# Result: Service
# Summarize: """
base_prompt = "Summarize: "
print(base_prompt)

Summarize: 


## Bart-Large-CNN

In [101]:
bart_large_cnn_tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
bart_large_cnn_model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")

In [102]:
bart_large_cnn_result = {"input":[], "summary":[], "restaurant": []}

In [103]:
for resto in df['resto_name'].unique():
    text_inputs = [rev for rev in df[df['resto_name'] == resto]['review_text']]
    
    prompt = base_prompt
    for text in tqdm(text_inputs):
        prompt += text + "\n"
    
    text_chunks = [prompt[i:i + 512] for i in range(0, len(prompt), 512)]
    full_summary = ""
    
    for chunk in text_chunks:
        inputs = bart_large_cnn_tokenizer(chunk, return_tensors='pt', max_length=50, truncation=True).to(device)
        
        with torch.no_grad():
            outputs = bart_large_cnn_model.generate(**inputs, max_length=50, min_length=10, 
                                                    length_penalty=2.0, num_beams=4, 
                                                    early_stopping=True)
        
        full_summary += bart_large_cnn_tokenizer.decode(outputs[0], skip_special_tokens=True).strip() + " "
    
    bart_large_cnn_result['input'].append(prompt)
    bart_large_cnn_result['summary'].append(full_summary)
    bart_large_cnn_result['restaurant'].append(resto)

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

  0%|          | 0/34 [00:00<?, ?it/s]

In [104]:
bart_large_cnn_rougescore = rouge_score.compute(references=bart_large_cnn_result['input'], predictions=bart_large_cnn_result['summary'])
bart_large_cnn_rougescore

{'rouge1': 0.4257642826262421,
 'rouge2': 0.4000844105113446,
 'rougeL': 0.4086790340182853,
 'rougeLsum': 0.4156824153163435}

In [105]:
bart_large_cnn_bertscore = bert_score.compute(references=bart_large_cnn_result['input'], predictions=bart_large_cnn_result['summary'], model_type="distilbert-base-uncased")

In [106]:
for k, v in bart_large_cnn_bertscore.items():
    if k=='hashcode':
        break
    average_score = np.average(v)
    print(k + ": " + str(average_score))

precision: 0.8854509166308812
recall: 0.8411748664719718
f1: 0.8619321669851031


In [107]:
bart_large_cnn_result_df = pd.DataFrame(bart_large_cnn_result)
bart_large_cnn_result_df.to_csv('bart_large_cnn_result.csv', index=None)

## T5 - Small

In [10]:
t5_small_tokenizer = AutoTokenizer.from_pretrained("google-t5/t5-small")
t5_small_model = AutoModelForSeq2SeqLM.from_pretrained("google-t5/t5-small")

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [34]:
t5_small_result = {"input":[], "summary":[], "restaurant": []}

In [123]:
for resto in df['resto_name'].unique():
    text_inputs = [rev for rev in df[df['resto_name'] == resto]['review_text']]
    
    prompt = base_prompt
    for text in tqdm(text_inputs):
        prompt += text + "\n"
    
    
    # inputs = t5_small_tokenizer(prompt, return_tensors='pt', max_length=512, truncation=True).to(device)
    
    # with torch.no_grad():
    #     outputs = t5_small_model.generate(**inputs, max_length=50, min_length=10, 
    #                                             length_penalty=2.0, num_beams=4, 
    #                                             early_stopping=True)
    
    # full_summary = t5_small_tokenizer.decode(outputs[0], skip_special_tokens=True).strip() + " "
    text_chunks = [prompt[i:i + 512] for i in range(0, len(prompt), 512)]
    full_summary = ""
    
    for chunk in text_chunks:
        inputs = t5_small_tokenizer(chunk, return_tensors='pt', max_length=50, truncation=True).to(device)
        
        with torch.no_grad():
            outputs = t5_small_model.generate(**inputs, max_length=50, min_length=10, 
                                                    length_penalty=2.0, num_beams=4, 
                                                    early_stopping=True)
        
        full_summary += t5_small_tokenizer.decode(outputs[0], skip_special_tokens=True).strip() + " "
    
    t5_small_result['input'].append(prompt)
    t5_small_result['summary'].append(full_summary)
    t5_small_result['restaurant'].append(resto)

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

  0%|          | 0/34 [00:00<?, ?it/s]

In [124]:
t5_small_result['input'][-1]

"Summarize: food was nice but please use to send one to two slices of onion/ some salad also and please don't use to make delay\nIt was amazing\nawesome taste\nfried rice was not good, no taste and mire oil was there.\nPackaging and Taste was good..\nWow! First of all, it was so different from the usual paneer and mushroom dishes, and the taste was simply outstanding!\nitems were missimg from ordered food.\ntoooo much oil in the sabzi,, bhai cholesterol ka patient banke manage kya?????? khud ke ghar mein bhi aise hi oil ki sabzi bnate? dear chef?\nthe quantity was too less at this prize way too much vegitables in noodles totally disgusting.\ngood combo amount of rice should be increased a bit\nbest Ambience best food🍟🍔🌭🥙🌯🍻🥂\nNestled in the heart of Varanasi, De Once More Cafe & Restaurant offers an exquisite dining experience that delights the senses. From the moment you walk in, the warm ambiance and attentive staff set the tone for an unforgettable evening.Ambiance:The decor strikes 

In [125]:
t5_small_result['summary'][-1]

"Summarize: food was nice but please use to send one to two slices of onion/ some salad also and please don't use to make delay It was amazing awesome taste fried rice was not good, no taste and mire oil ! i oil ki sabzi bnate? dear chef? dear chef? the quantity was too less at this prize way too much vegitables in noodles totally disgusting. :The service offers a warm atmosphere with gance and cozy charm, with soft lighting and tasteful art adorning the walls.Service:The service is extremely inviting. .The spring roll was also very less Rely superb teste cold drink was not provided Pasta was good. Though I expected a little bit more mushroom in it.The spring roll was also very less Rely superb teste cold is not fresh...as soon as i opened the bottle it smelled so bad Food quality is best but quantity is very small according to money best thali I have ever had. best quality food but portion size is "

In [119]:
t5_small_rougescore = rouge_score.compute(references=t5_small_result['input'], predictions=t5_small_result['summary'])
t5_small_rougescore

{'rouge1': 0.11506137853502635,
 'rouge2': 0.10921263619615672,
 'rougeL': 0.10965198156497236,
 'rougeLsum': 0.11324412800805071}

## Pegasus

In [113]:
pegasus_tokenizer = AutoTokenizer.from_pretrained("google/bigbird-pegasus-large-arxiv")
pegasus_model = AutoModelForSeq2SeqLM.from_pretrained("google/bigbird-pegasus-large-arxiv")

In [114]:
pegasus_result = {"input":[], "summary":[], "restaurant": []}

In [115]:
for resto in df['resto_name'].unique():
    text_inputs = [rev for rev in df[df['resto_name'] == resto]['review_text']]
    
    prompt = base_prompt
    for text in tqdm(text_inputs):
        prompt += text + "\n"
    
    
    inputs = pegasus_tokenizer(prompt, return_tensors='pt', max_length=512, truncation=True).to(device)
    
    with torch.no_grad():
        outputs = pegasus_model.generate(**inputs, max_length=50, min_length=10, 
                                                length_penalty=2.0, num_beams=4, 
                                                early_stopping=True)
    
    full_summary = pegasus_tokenizer.decode(outputs[0], skip_special_tokens=True).strip() + " "
    
    pegasus_result['input'].append(prompt)
    pegasus_result['summary'].append(full_summary)
    pegasus_result['restaurant'].append(resto)

  0%|          | 0/50 [00:00<?, ?it/s]

Attention type 'block_sparse' is not possible if sequence_length: 512 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_full'...


model.safetensors:   0%|          | 0.00/2.31G [00:00<?, ?B/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

  0%|          | 0/34 [00:00<?, ?it/s]

In [116]:
pegasus_result['summary'][0]

'we have beenhendi at the fusion oriental restaurant .<n> we have been to many places but this is by far the best meal we have ever experienced .<n> the staff are very friendly and very accommodating .<n> the food is equally as '

In [120]:
pegasus_rougescore = rouge_score.compute(references=pegasus_result['input'], predictions=pegasus_result['summary'])
pegasus_rougescore

{'rouge1': 0.062023017391476934,
 'rouge2': 0.005571326456446445,
 'rougeL': 0.048544605457600754,
 'rougeLsum': 0.05339531178439427}

# Model yang Dipilih
Model yang digunakan adalah T5 - Small karena kemampuan yang dapat digunakan untuk melakukan summary secara abstractive.

In [8]:
df_sentiment = pd.read_csv('/kaggle/input/zomatoreview/deberta-v3-base-absa-v1.csv')
df_sentiment.head()

,Unnamed: 0,text,category,topic_food,score_food,topic_place,score_place,topic_price,score_price,topic_service,score_service,topic_count,main_topics,primary_topic,primary_score,sentiment_food,sentiment_place,sentiment_price,sentiment_service
0,0,"To be completely fair, the only redeeming fact...",food,1,0.223695,0,0.000000,0,0.0,0,0.0,1,food,food,0.223695,Positive,Not Found,Not Found,Not Found
1,1,"The food is uniformly exceptional, with a very...",food,1,0.515413,1,0.378481,0,0.0,0,0.0,2,"food, place",food,0.515413,Positive,Neutral,Not Found,Not Found
2,2,"Not only was the food outstanding, but the lit...",food,1,0.330244,0,0.000000,0,0.0,0,0.0,1,food,food,0.330244,Positive,Not Found,Not Found,Not Found
3,3,It is very overpriced and not very tasty.,food,1,0.371381,0,0.000000,0,0.0,0,0.0,1,food,food,0.371381,Negative,Not Found,Not Found,Not Found
4,4,Our agreed favorite is the orrechiete with sau...,food,1,0.424373,1,0.321227,0,0.0,0,0.0,2,"food, place",food,0.424373,Neutral,Neutral,Not Found,Not Found


In [21]:
result = {"category":[], "sentiment":[], "summary":[]}

In [22]:
for category in df_sentiment['category'].unique():
    # Get each sentiment for the category
    print("Category " + category)
    for sentiment in ['Positive', 'Negative', 'Neutral']:
        print("Sentiment " + sentiment)
        prompt = "Summarize: "
        
        for text in tqdm(df_sentiment[df_sentiment['sentiment_' + category] == sentiment]['text']):
            prompt += text + "\n"
            
        # text_chunks = [prompt[i:i + 512] for i in range(0, len(prompt), 512)]
        full_summary = ""
        
        # for chunk in text_chunks:
        inputs = t5_small_tokenizer(prompt, return_tensors='pt', max_length=512, truncation=True).to(device)
        
        with torch.no_grad():
            outputs = t5_small_model.generate(**inputs, max_length=50, min_length=10, 
                                                    length_penalty=2.0, num_beams=4, 
                                                    early_stopping=True)
        
        full_summary += t5_small_tokenizer.decode(outputs[0], skip_special_tokens=True).strip() + " "
        print(full_summary)

        result['category'].append(category)
        result['sentiment'].append(sentiment)
        result['summary'].append(full_summary)

Category food
Sentiment Positive


  0%|          | 0/515 [00:00<?, ?it/s]

: To be completely fair, the only redeeming factor was the food, which was above average, but couldn't make up for all the other deficiencies of Teodora. The food is uniformly exceptional, with a very 
Sentiment Negative


  0%|          | 0/137 [00:00<?, ?it/s]

: It is very overpriced and not very tasty. They did not have mayonnaise, forgot our toast, left out ingredients (ie cheese in an omelet), below hot temperatures and the bacon was so over cooked 
Sentiment Neutral


...

,,, but none went to the chefs. From the incredible food, to the warm atmosphere, to the friendly service, this downtown neighborhood spot doesn't miss a beat. It's an excellent place to have 
Sentiment Negative


  0%|          | 0/145 [00:00<?, ?it/s]

the menu is very limited - we counted 4 or 5 entrees. Unfortunately, the food is outstanding, but everything else about this restaurant is the pits. The food is terrible and overall, I would have 
Sentiment Neutral


  0%|          | 0/451 [00:00<?, ?it/s]

the food is uniformly exceptional, with a very capable kitchen which will proudly whip up whatever you feel like eating. The food is uniformly exceptional, with a very capable kitchen which will proudly whip up whatever you feel 


In [23]:
final_df = pd.DataFrame(result, index=None)
final_df.to_csv('result.csv')